# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../downloads/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,louisville,38.2542,-85.7594,275.11,80,0,2.06,US,1701915953
1,1,adamstown,-25.0660,-130.1015,296.14,79,96,5.52,PN,1701915823
2,2,tura,25.5198,90.2201,290.22,92,100,0.92,IN,1701916087
3,3,bilibino,68.0546,166.4372,241.58,100,100,1.74,RU,1701915833
4,4,hithadhoo,-0.6000,73.0833,301.04,79,39,4.99,MV,1701916088


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
%%capture --no-display

# Configure the map plot
map_plot_cities = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    size='Humidity',
    tiles="EsriImagery",
    frame_width=800,
    frame_height=600,
    alpha=1,
    scale=1,
    color="City",
    hover_cols=["City", "Humidity"]
)
# Display the map
map_plot_cities

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = {
    'Max Temp': (21, 27),  
    'Humidity': (0, 120),       
    'Wind Speed': (0, 4.5),      
    'Cloudiness': (0, 0)      
}

filtered_df = city_data_df[
    (city_data_df['Max Temp'].between(ideal_conditions['Max Temp'][0], ideal_conditions['Max Temp'][1])) &
    (city_data_df['Humidity'].between(ideal_conditions['Humidity'][0], ideal_conditions['Humidity'][1])) &
    (city_data_df['Wind Speed'].between(ideal_conditions['Wind Speed'][0], ideal_conditions['Wind Speed'][1])) &
    (city_data_df['Cloudiness'].between(ideal_conditions['Cloudiness'][0], ideal_conditions['Cloudiness'][1]))
]

# Drop any rows with null values
filtered_df = filtered_df.dropna()

# Display sample data
filtered_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
81,81,makakilo city,21.3469,-158.0858,21.66,81,0,2.57,US,1666108282
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,
51,hilo,US,19.7297,-155.0900,83,
63,banda,IN,25.4833,80.3333,52,
81,makakilo city,US,21.3469,-158.0858,81,
152,kahului,US,20.8947,-156.4700,60,
197,gat,IL,31.6100,34.7642,100,
211,laguna,US,38.4210,-121.4238,79,
240,tikaitnagar,IN,26.9500,81.5833,59,
265,san quintin,MX,30.4833,-115.9500,74,
340,santa rosalia,MX,27.3167,-112.2833,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'radius': radius,
    'limit': 1,  # Retrieve only the first result
    'apiKey': geoapify_key,  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude, longitude = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(10)

Starting hotel search
kapaa - nearest hotel: Sheraton Kauai Coconut Beach Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Four Seasons O'ahu at Ko Olina
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: TownePlace Suites Sacramento Elk Grove
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Don Eddies Landing
santa rosalia - nearest hotel: Hotel Francés
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: Hospitality Port Hedland
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Santa Lucia
stolac - nearest hotel: Hotel Stolac
guerrero negro - nearest hotel: Cabanas Malarrimo


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Sheraton Kauai Coconut Beach Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Four Seasons O'ahu at Ko Olina
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,TownePlace Suites Sacramento Elk Grove
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Don Eddies Landing
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel Francés


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="EsriImagery",
    frame_width=800,
    frame_height=600,
    size=50, 
    color="City",
    hover_cols=["City", "Humidity", "Hotel Name", "Country"]
)

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)